# Environment Setup and Configuration


### 🚀 The Pre-Flight Checklist**

Alright, before we get to the cool machine learning magic, we have to do the boring-but-brilliant setup.

This next cell is our project's bouncer. It checks our Python environment to see what tools we're working with and peeks in the back to see if a GPU is ready to do the heavy lifting. It then pulls up our "recipe book" of settings and keeps a to-do list to track our progress.

Basically, it makes sure all systems are go before we launch. If it runs without a peep, we're ready for the fun stuff.

In [1]:
import sys
import os
import platform
import importlib.util
import importlib.metadata
from dataclasses import dataclass, field
from typing import List, Dict, Any

# Global pipeline state management
@dataclass
class PipelineState:
    valid: bool = True
    skip_reason: str = ""
    data_loaded: bool = False
    model_trained: bool = False
    predictions_generated: bool = False
    
    def invalidate(self, reason: str):
        self.valid = False
        self.skip_reason = reason
        print(f"⚠️ Pipeline halted: {reason}")

# Initialize global pipeline state
pipeline_state = PipelineState()

def env_check():
    """
    Checks the user's environment for required and optional packages,
    and sets global flags to control the notebook's behavior.
    """
    specs = {
        # Core
        "torch": "2.0.0", "numpy": "1.23.5", "pandas": "2.0.0", "scipy": "1.10.0",
        # Optional ML/DL
        "transformers": "4.30.0", "datasets": "2.14.0", "accelerate": "0.21.0",
        "bitsandbytes": "0.41.0", "sentence-transformers": "2.2.2",
        "tqdm": "4.64.1", "sklearn": "1.2.2",
        # Optional Viz/UI
        "matplotlib": "3.7.1", "ipywidgets": "8.0.4",
        # Less common but useful
        "holidays": "0.29", "timm": "0.9.5", "opencv-python": "4.8.0.74",
        "pytorch_lightning": "2.0.0"
    }

    print("--- Environment & Library Check ---")
    detected_versions = {}
    flags = {}
    optional_installs = []

    for pkg, min_version in specs.items():
        try:
            version = importlib.metadata.version(pkg)
            detected_versions[pkg] = version
            flags[f"USE_{pkg.upper().replace('-', '_')}"] = True
        except importlib.metadata.PackageNotFoundError:
            flags[f"USE_{pkg.upper().replace('-', '_')}"] = False
            if pkg in ["transformers", "accelerate", "bitsandbytes", "ipywidgets", "holidays", "tqdm", "scikit-learn"]:
                install_name = "scikit-learn" if pkg == "sklearn" else pkg
                optional_installs.append(f"pip install {install_name}")

    use_torch = flags.get('USE_TORCH', False)
    if use_torch:
        import torch
        flags['CUDA_AVAILABLE'] = torch.cuda.is_available()
    else:
        flags['CUDA_AVAILABLE'] = False
    
    print("✅ Environment check complete.")
    if optional_installs:
        print("💡 Optional libraries not found. For enhanced features, consider installing:")
        for cmd in set(optional_installs):
            print(f"   {cmd}")
    return flags

@dataclass
class ProjectConfig:
    env_flags: Dict[str, bool] = field(default_factory=dict)
    DATA_PATH: str = "./data_cache"
    TARGET_COLUMN: str = "target_load"
    TIME_COLUMN: str = "utc_timestamp"
    GRANULARITY: str = "H"
    MODEL_NAME: str = "tcn_transformer_hybrid"
    LOOKBACK_WINDOW: int = 72
    FORECAST_HORIZON: int = 12
    QUANTILES: List[float] = field(default_factory=lambda: [0.1, 0.5, 0.9])
    BATCH_SIZE: int = 64
    EPOCHS: int = 10
    LEARNING_RATE: float = 1e-4
    PATIENCE: int = 3
    USE_AMP: bool = True
    MODEL_SAVE_PATH: str = "./models"

    def __post_init__(self):
        os.makedirs(self.DATA_PATH, exist_ok=True)
        os.makedirs(self.MODEL_SAVE_PATH, exist_ok=True)
        self.USE_AMP = self.env_flags.get('CUDA_AVAILABLE', False)

def get_progress_bar():
    """
    Returns the tqdm progress bar if available, otherwise a dummy iterator.
    """
    if config.env_flags.get('USE_TQDM', False):
        from tqdm.autonotebook import tqdm
        return tqdm
    else:
        # Fallback to a dummy iterator that just returns the object
        return lambda x, *args, **kwargs: x

# Run the check and create the config object
try:
    ENV_FLAGS = env_check()
    config = ProjectConfig(env_flags=ENV_FLAGS)
    print("✅ Configuration initialized successfully.")
except Exception as e:
    pipeline_state.invalidate(f"Environment setup failed: {str(e)}")

--- Environment & Library Check ---
✅ Environment check complete.
✅ Configuration initialized successfully.


# Data Access Layer
### 💾 Time to Get the Data

Alright, let's grab some data. This part is all about fetching our datasets from the internet.

* This block is smart about it, it will only download the data once and then save a local copy (a "cache"). The next time you run it, it'll be lightning-fast because it'll just load the data from your computer. We're also doing a quick cleanup to make sure the column names are consistent, so we don't have to worry about it later.

In [2]:
import pandas as pd
import os

def check_pipeline_state(operation_name):
    """Check if pipeline is valid before proceeding with operation"""
    if not pipeline_state.valid:
        print(f"⚠️ Skipping {operation_name}: {pipeline_state.skip_reason}")
        return False
    return True

def load_opsd(cache_path=None, resample_freq=None):
    """
    Loads the Open Power System Data (OPSD) for Germany.
    Caches the data with its original 'Consumption' column name.
    """
    if not check_pipeline_state("OPSD data loading"):
        return pd.DataFrame()
        
    if cache_path is None:
        cache_path = config.DATA_PATH
    if resample_freq is None:
        resample_freq = config.GRANULARITY
        
    file_path = os.path.join(cache_path, f"opsd_germany_{resample_freq}_original.pkl")
    if os.path.exists(file_path):
        try:
            df = pd.read_pickle(file_path)
            print("✅ Loaded cached OPSD data")
            return df
        except Exception as e:
            print(f"Warning: Could not load cached data: {e}")
    
    print("⬇ Downloading OPSD data...")
    url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
    try:
        df = pd.read_csv(url, index_col='Date', parse_dates=True)
        if 'Consumption' not in df.columns:
            raise ValueError("Expected 'Consumption' column not found in OPSD data")
        
        df = df[['Consumption']].resample(resample_freq).ffill()
        df.to_pickle(file_path)
        print(f"💾 Cached OPSD data to {file_path}")
        return df
    except Exception as e:
        pipeline_state.invalidate(f"Error downloading or processing OPSD data: {e}")
        return pd.DataFrame()

def load_uci_household(cache_path=None, resample_freq=None):
    """
    Loads the UCI Individual household electric power consumption dataset.
    Caches the data with its original 'Global_active_power' column name.
    """
    if not check_pipeline_state("UCI household data loading"):
        return pd.DataFrame()
        
    if cache_path is None:
        cache_path = config.DATA_PATH
    if resample_freq is None:
        resample_freq = config.GRANULARITY
        
    file_path = os.path.join(cache_path, f"uci_household_{resample_freq}_original.pkl")
    if os.path.exists(file_path):
        try:
            df = pd.read_pickle(file_path)
            print("✅ Loaded cached UCI household data")
            return df
        except Exception as e:
            print(f"Warning: Could not load cached data: {e}")
    
    print("⬇ Downloading and processing UCI Household data...")
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip'
    try:
        df = pd.read_csv(url, sep=';', compression='zip', low_memory=False, 
                        na_values=['?'], parse_dates={'datetime': ['Date', 'Time']}, 
                        infer_datetime_format=True, dayfirst=True)
        df = df.set_index('datetime')
        if 'Global_active_power' not in df.columns:
            raise ValueError("Expected 'Global_active_power' column not found in UCI data")
            
        df = df[['Global_active_power']].astype(float)
        df = df.resample(resample_freq).sum()
        df.to_pickle(file_path)
        print(f"💾 Cached UCI data to {file_path}")
        return df
    except Exception as e:
        pipeline_state.invalidate(f"Error downloading or processing UCI household data: {e}")
        return pd.DataFrame()

# Load and standardize data
if check_pipeline_state("Data loading"):
    print("--- Data Access Layer ---")
    
    # Load the raw data (default to OPSD)
    try:
        df_opsd_raw = load_opsd()
        
        if not df_opsd_raw.empty and 'Consumption' in df_opsd_raw.columns:
            # Standardize the column name
            df_opsd = df_opsd_raw.rename(columns={'Consumption': config.TARGET_COLUMN})
            pipeline_state.data_loaded = True
            
            print("\nOPSD Data Sample (standardized):")
            print(df_opsd.head())
            print(f"Data shape: {df_opsd.shape}")
            print(f"Date range: {df_opsd.index.min()} to {df_opsd.index.max()}")
        else:
            pipeline_state.invalidate("Failed to load OPSD data or missing expected columns")
            df_opsd = pd.DataFrame()
            
    except Exception as e:
        pipeline_state.invalidate(f"Error in data loading process: {str(e)}")
        df_opsd = pd.DataFrame()

--- Data Access Layer ---
✅ Loaded cached OPSD data

OPSD Data Sample (standardized):
                     target_load
Date                            
2006-01-01 00:00:00     1069.184
2006-01-01 01:00:00     1069.184
2006-01-01 02:00:00     1069.184
2006-01-01 03:00:00     1069.184
2006-01-01 04:00:00     1069.184
Data shape: (105169, 1)
Date range: 2006-01-01 00:00:00 to 2017-12-31 00:00:00


# Data Cleaning and Preprocessing
### 💪 Whipping the Data into Shape

* Raw data is a bit of a wild beast. It's messy, has weird gaps, and sometimes throws out crazy, unbelievable numbers. You can't just feed that to a sophisticated model; it would get indigestion.

* This next cell is the data's personal trainer. It forces our dataset to do some cleanup:

* Fills in the Blanks: Patches up any missing values.

* Trims the Fat: Clips the extreme, wild outliers that don't make sense.

* Puts Everyone on the Same Scale: Resizes all the numbers so they live in the same neighborhood.

This way, our model can focus on the real patterns instead of getting distracted by a few loud, obnoxious numbers.

In [3]:
import numpy as np

# Check if scikit-learn is available for the scaler
if config.env_flags.get('USE_SKLEARN', False):
    from sklearn.preprocessing import StandardScaler

def preprocess_for_model(df, target_col):
    """
    Cleans, scales, and prepares the dataframe for model input.
    """
    if not check_pipeline_state("Data preprocessing"):
        return pd.DataFrame(), None
        
    if df.empty:
        pipeline_state.invalidate("Cannot preprocess empty dataframe")
        return pd.DataFrame(), None
        
    if target_col not in df.columns:
        pipeline_state.invalidate(f"Target column '{target_col}' not found in dataframe")
        return pd.DataFrame(), None
    
    try:
        print("\n--- Preprocessing Data ---")
        
        # Make a copy to avoid modifying original data
        df_processed = df.copy()
        
        # 1. Handle missing values
        original_rows = len(df_processed)
        df_processed = df_processed.dropna(subset=[target_col])
        if len(df_processed) < original_rows:
            print(f"   - Dropped {original_rows - len(df_processed)} rows with missing target.")
        
        # Check if we have enough data after dropping missing values
        if len(df_processed) < config.LOOKBACK_WINDOW + config.FORECAST_HORIZON:
            pipeline_state.invalidate(f"Insufficient data after preprocessing: {len(df_processed)} rows, need at least {config.LOOKBACK_WINDOW + config.FORECAST_HORIZON}")
            return pd.DataFrame(), None
        
        # Use time-based interpolation for missing values in other columns if any
        df_processed = df_processed.interpolate(method='time')
        
        # 2. Outlier handling (simple clipping)
        q1 = df_processed[target_col].quantile(0.01)
        q99 = df_processed[target_col].quantile(0.99)
        df_processed[target_col] = np.clip(df_processed[target_col], q1, q99)
        print("   - Clipped target values to 1st and 99th percentiles.")

        # 3. Scaling
        if config.env_flags.get('USE_SKLEARN', False):
            scaler = StandardScaler()
            df_processed[f"{target_col}_scaled"] = scaler.fit_transform(df_processed[[target_col]])
            print("   - Scaled data using StandardScaler.")
        else:
            # Fallback to manual scaling
            mean = df_processed[target_col].mean()
            std = df_processed[target_col].std()
            if std == 0:
                pipeline_state.invalidate("Target column has zero standard deviation - cannot scale")
                return pd.DataFrame(), None
            df_processed[f"{target_col}_scaled"] = (df_processed[target_col] - mean) / std
            scaler = {'mean': mean, 'std': std}  # Save for inverse transform
            print("   - Scaled data manually (NumPy/Pandas fallback).")
        
        return df_processed, scaler
        
    except Exception as e:
        pipeline_state.invalidate(f"Error during preprocessing: {str(e)}")
        return pd.DataFrame(), None

# Execute preprocessing
if check_pipeline_state("Preprocessing execution") and pipeline_state.data_loaded:
    try:
        data_df, scaler = preprocess_for_model(df_opsd.copy(), config.TARGET_COLUMN)
        
        if not data_df.empty and scaler is not None:
            print("\nPreprocessed Data Sample:")
            print(data_df.head())
            print(f"Processed data shape: {data_df.shape}")
        else:
            pipeline_state.invalidate("Preprocessing failed to produce valid output")
            
    except Exception as e:
        pipeline_state.invalidate(f"Preprocessing execution failed: {str(e)}")
        data_df, scaler = pd.DataFrame(), None
else:
    data_df, scaler = pd.DataFrame(), None


--- Preprocessing Data ---
   - Clipped target values to 1st and 99th percentiles.
   - Scaled data manually (NumPy/Pandas fallback).

Preprocessed Data Sample:
                     target_load  target_load_scaled
Date                                                
2006-01-01 00:00:00     1069.184           -1.637032
2006-01-01 01:00:00     1069.184           -1.637032
2006-01-01 02:00:00     1069.184           -1.637032
2006-01-01 03:00:00     1069.184           -1.637032
2006-01-01 04:00:00     1069.184           -1.637032
Processed data shape: (105169, 2)


# Feature Engineering
### 🕵️‍♂️ Giving Our Data Superpowers

* Our data is clean, but it's a bit plain. A machine learning model needs more than just a single column of numbers; it needs clues to find patterns. This process, called feature engineering, is like giving our data a detective's toolkit.

* What Clues Are We Creating?
We're adding several new columns to give the model more context about each data point:

* Time Features: We're telling the model the hour, day of the week, and month for each entry. This helps it learn daily, weekly, and yearly cycles.

* Holiday Flags: We mark down whether a day is a holiday, since people's behavior (and energy use) changes dramatically on those days.

* Lag Features: We add columns showing what the value was 24 hours ago, 48 hours ago, and a week ago. This gives the model a sense of recent history.

* Rolling Averages: We calculate the average and standard deviation over the last day and week. This tells the model if things have been trending up, down, or have been unusually volatile.

* Fourier Terms: These are fancy sin/cos waves that help the model understand the smooth, cyclical nature of a year, preventing it from thinking December 31st is wildly different from January 1st.

In [4]:
if config.env_flags.get('USE_HOLIDAYS', False):
    import holidays

def build_features(df, target_col):
    """
    Enriches the dataframe with time-based, holiday, and lagged features.
    """
    if not check_pipeline_state("Feature engineering"):
        return pd.DataFrame()
        
    if df.empty:
        pipeline_state.invalidate("Cannot build features from empty dataframe")
        return pd.DataFrame()
        
    if target_col not in df.columns:
        pipeline_state.invalidate(f"Target column '{target_col}' not found for feature engineering")
        return pd.DataFrame()
    
    try:
        print("\n--- Building Features ---")
        
        # Make a copy to avoid modifying original data
        df_featured = df.copy()
        
        # Time-based features
        df_featured['hour'] = df_featured.index.hour
        df_featured['dayofweek'] = df_featured.index.dayofweek
        df_featured['month'] = df_featured.index.month
        df_featured['year'] = df_featured.index.year
        df_featured['dayofyear'] = df_featured.index.dayofyear
        print("   - Added time-based features (hour, dayofweek, etc.).")

        # Holiday features
        if config.env_flags.get('USE_HOLIDAYS', False):
            try:
                de_holidays = holidays.Germany()
                df_featured['is_holiday'] = df_featured.index.map(lambda x: 1 if x in de_holidays else 0)
                print("   - Added holiday features using 'holidays' library.")
            except Exception as e:
                print(f"   - Warning: Could not use holidays library ({e}), using fallback.")
                df_featured['is_holiday'] = ((df_featured.index.month == 12) & (df_featured.index.day.isin([24, 25, 26, 31]))) | \
                                           ((df_featured.index.month == 1) & (df_featured.index.day == 1))
                df_featured['is_holiday'] = df_featured['is_holiday'].astype(int)
                print("   - Added simple holiday fallback (e.g., Christmas/New Year).")
        else:
            # Simple fallback for major holidays
            df_featured['is_holiday'] = ((df_featured.index.month == 12) & (df_featured.index.day.isin([24, 25, 26, 31]))) | \
                                       ((df_featured.index.month == 1) & (df_featured.index.day == 1))
            df_featured['is_holiday'] = df_featured['is_holiday'].astype(int)
            print("   - Added simple holiday fallback (e.g., Christmas/New Year).")

        # Lagged features
        for lag in [24, 48, 168]:  # 1 day, 2 days, 1 week ago
            df_featured[f'lag_{lag}'] = df_featured[target_col].shift(lag)
        print("   - Added lagged features (24h, 48h, 168h).")

        # Rolling window features
        for window in [24, 168]:
            df_featured[f'rolling_mean_{window}'] = df_featured[target_col].shift(1).rolling(window=window).mean()
            df_featured[f'rolling_std_{window}'] = df_featured[target_col].shift(1).rolling(window=window).std()
        print("   - Added rolling window features (mean/std over 24h, 168h).")

        # Fourier terms for seasonality
        df_featured['sin_dayofyear'] = np.sin(2 * np.pi * df_featured['dayofyear'] / 365.25)
        df_featured['cos_dayofyear'] = np.cos(2 * np.pi * df_featured['dayofyear'] / 365.25)
        print("   - Added Fourier terms for yearly seasonality.")

        # Drop rows with NaNs created by lags/rolling windows
        initial_rows = len(df_featured)
        df_featured = df_featured.dropna()
        dropped_rows = initial_rows - len(df_featured)
        
        if dropped_rows > 0:
            print(f"   - Dropped {dropped_rows} rows with NaN values from feature engineering.")
        
        # Check if we still have enough data
        min_required = config.LOOKBACK_WINDOW + config.FORECAST_HORIZON
        if len(df_featured) < min_required:
            pipeline_state.invalidate(f"Insufficient data after feature engineering: {len(df_featured)} rows, need at least {min_required}")
            return pd.DataFrame()
        
        print(f"   - Final dataset shape: {df_featured.shape}")
        return df_featured
        
    except Exception as e:
        pipeline_state.invalidate(f"Error during feature engineering: {str(e)}")
        return pd.DataFrame()

# Execute feature engineering
if check_pipeline_state("Feature engineering execution") and not data_df.empty:
    try:
        featured_df = build_features(data_df, f"{config.TARGET_COLUMN}_scaled")
        
        if not featured_df.empty:
            print("\nData Sample with Features:")
            print(featured_df.head())
            print(f"Feature columns: {list(featured_df.columns)}")
        else:
            pipeline_state.invalidate("Feature engineering failed to produce valid output")
            
    except Exception as e:
        pipeline_state.invalidate(f"Feature engineering execution failed: {str(e)}")
        featured_df = pd.DataFrame()
else:
    featured_df = pd.DataFrame()


--- Building Features ---
   - Added time-based features (hour, dayofweek, etc.).
   - Added holiday features using 'holidays' library.
   - Added lagged features (24h, 48h, 168h).
   - Added rolling window features (mean/std over 24h, 168h).
   - Added Fourier terms for yearly seasonality.
   - Dropped 168 rows with NaN values from feature engineering.
   - Final dataset shape: (105001, 17)

Data Sample with Features:
                     target_load  target_load_scaled  hour  dayofweek  month  \
Date                                                                           
2006-01-08 00:00:00     1207.985           -0.794307     0          6      1   
2006-01-08 01:00:00     1207.985           -0.794307     1          6      1   
2006-01-08 02:00:00     1207.985           -0.794307     2          6      1   
2006-01-08 03:00:00     1207.985           -0.794307     3          6      1   
2006-01-08 04:00:00     1207.985           -0.794307     4          6      1   

               

# Splitting and Walk-Forward Validation

### ⏳ The Time Traveler's Test: Splitting Our Data

When you're predicting the future, you can't cheat.
A normal train_test_split would shuffle our data, letting the model peek at future events to "predict" the past. That's like giving it a crystal ball—it'll look like a genius, but it's completely useless in the real world.

The Fair Fight: Walk-Forward Validation
Instead, we use a much smarter method called Walk-Forward Validation. It mimics reality by slicing up our timeline into a series of "past vs. future" challenges.

It works like this:

* Train the model on data from the past (e.g., Week 1-4).

* Test it by having it predict the immediate future (e.g., Week 5).

Then, we "walk forward" and repeat: train on Week 1-5, test on Week 6.

Our walk_forward_split function is the machine that creates these realistic training and testing windows. This ensures our model is judged on its ability to genuinely forecast, with no cheating allowed.

In [5]:
def walk_forward_split(data, n_splits=5, lookback=None, horizon=None):
    """
    Generates indices for walk-forward validation.
    Returns None if data is insufficient.
    """
    if not check_pipeline_state("Walk-forward split"):
        return None
        
    if data.empty:
        pipeline_state.invalidate("Cannot create splits from empty data")
        return None
        
    if lookback is None:
        lookback = config.LOOKBACK_WINDOW
    if horizon is None:
        horizon = config.FORECAST_HORIZON
    
    try:
        total_samples = len(data)
        min_required = lookback + horizon
        
        # Check if we have enough data for even one split
        if total_samples < min_required:
            pipeline_state.invalidate(f"Insufficient data for walk-forward split: {total_samples} samples, need at least {min_required}")
            return None
        
        # Calculate maximum possible splits
        max_possible_splits = total_samples - min_required + 1
        actual_splits = min(n_splits, max_possible_splits)
        
        if actual_splits < n_splits:
            print(f"   - Requested {n_splits} splits, but only {actual_splits} possible with available data")
        
        # Start splitting from the end of the data
        split_points = np.linspace(
            total_samples - actual_splits * horizon, 
            total_samples - horizon, 
            actual_splits, 
            dtype=int
        )

        valid_splits = []
        for split_point in split_points:
            train_end = split_point
            train_start = max(0, train_end - lookback * 10)  # Limit training history size for efficiency
            test_start = train_end
            test_end = test_start + horizon
            
            if test_end <= total_samples and train_end > train_start:
                train_indices = np.arange(train_start, train_end)
                test_indices = np.arange(test_start, test_end)
                valid_splits.append((train_indices, test_indices))
        
        if not valid_splits:
            pipeline_state.invalidate("No valid splits could be created")
            return None
            
        return valid_splits
        
    except Exception as e:
        pipeline_state.invalidate(f"Error creating walk-forward splits: {str(e)}")
        return None

# Create splits and validate
if check_pipeline_state("Split creation") and not featured_df.empty:
    print("\n--- Walk-Forward Validation Splits ---")
    
    try:
        splits = walk_forward_split(featured_df)
        
        if splits is not None and len(splits) > 0:
            print(f"Created {len(splits)} valid splits")
            
            # Show first few splits for inspection
            for i, (train_indices, test_indices) in enumerate(splits[:3]):  # Show first 3 splits
                print(f"Split {i+1}:")
                print(f"  Train: indices from {train_indices.min()} to {train_indices.max()} (length {len(train_indices)})")
                print(f"  Test:  indices from {test_indices.min()} to {test_indices.max()} (length {len(test_indices)})")
            
            if len(splits) > 3:
                print(f"  ... and {len(splits) - 3} more splits")
                
            # Store the first split for model training
            train_indices, test_indices = splits[0]
            
            # Validate split sizes
            if len(train_indices) < config.LOOKBACK_WINDOW:
                pipeline_state.invalidate(f"Training split too small: {len(train_indices)} < {config.LOOKBACK_WINDOW}")
            elif len(test_indices) < config.FORECAST_HORIZON:
                pipeline_state.invalidate(f"Test split too small: {len(test_indices)} < {config.FORECAST_HORIZON}")
            else:
                print("✅ Splits validated successfully")
                
        else:
            pipeline_state.invalidate("No valid splits were created")
            train_indices, test_indices = None, None
            
    except Exception as e:
        pipeline_state.invalidate(f"Error during split creation: {str(e)}")
        splits = None
        train_indices, test_indices = None, None
else:
    splits = None
    train_indices, test_indices = None, None


--- Walk-Forward Validation Splits ---
Created 5 valid splits
Split 1:
  Train: indices from 104221 to 104940 (length 720)
  Test:  indices from 104941 to 104952 (length 12)
Split 2:
  Train: indices from 104233 to 104952 (length 720)
  Test:  indices from 104953 to 104964 (length 12)
Split 3:
  Train: indices from 104245 to 104964 (length 720)
  Test:  indices from 104965 to 104976 (length 12)
  ... and 2 more splits
✅ Splits validated successfully


# TCN + Transformer Hybrid Model 
### 🧠 Building the Brain: A Hybrid TCN-Transformer

It's time to build our forecasting model! We're not just picking one off the shelf; we're building a powerful hybrid that combines the best of two worlds. Think of it as a super-team-up between two different kinds of geniuses.

**Meet the Team**
Our model, the TCNTransformerHybrid, is made of two main parts that work together:

* **The Local Expert (TCN):** First up is the Temporal Convolutional Network (TCN). This part is brilliant at spotting local patterns and short-term trends. It scans the sequence of data like a detective looking for immediate clues and textures in the timeline.

* **The Big Picture Thinker (Transformer):** The features identified by the TCN are then passed to a Transformer Encoder. The Transformer is a master of understanding context and relationships between points far apart in time. It figures out which past events are truly important for predicting the future, no matter how long ago they happened.

By combining them, the TCN handles the detailed, local feature extraction, and the Transformer handles the high-level, long-range dependencies. This powerful duo gives us a robust model capable of capturing complex time-series dynamics. Finally, a simple Linear layer takes this rich understanding and shapes it into our final forecast.

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size
        
    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class TCNTransformerHybrid(nn.Module):
    def __init__(self, input_size, output_size, horizon, num_quantiles, tcn_channels=[32, 64], d_model=64, nhead=4, num_layers=2):
        super(TCNTransformerHybrid, self).__init__()
        self.tcn = TemporalConvNet(input_size, tcn_channels)
        
        encoder_layers = nn.TransformerEncoderLayer(d_model=tcn_channels[-1], nhead=nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)
        
        self.output_size = output_size
        self.horizon = horizon
        self.num_quantiles = num_quantiles
        
        # Flattened output for horizon * quantiles
        self.fc = nn.Linear(tcn_channels[-1], output_size * horizon * num_quantiles)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        x = x.permute(0, 2, 1)  # TCN expects (batch, channels, seq_len)
        tcn_out = self.tcn(x)
        tcn_out = tcn_out.permute(0, 2, 1)  # Back to (batch, seq_len, features)
        
        transformer_out = self.transformer_encoder(tcn_out)
        
        # We only need the output from the last time step of the encoder
        last_step_out = transformer_out[:, -1, :]
        
        # Fully connected layer to get the forecast
        out = self.fc(last_step_out)
        
        # Reshape to (batch_size, horizon, num_quantiles)
        out = out.view(-1, self.horizon, self.num_quantiles)
        return out

# Initialize model architecture if pipeline is valid
if check_pipeline_state("Model definition") and not featured_df.empty and train_indices is not None:
    print("--- TCN+Transformer Hybrid Model Definition ---")
    
    try:
        # Calculate feature dimensions
        target_col_scaled = f"{config.TARGET_COLUMN}_scaled"
        feature_cols = [c for c in featured_df.columns if c not in [config.TARGET_COLUMN, target_col_scaled]]
        input_features = len(feature_cols)
        
        print(f"Input features: {input_features}")
        print(f"Feature columns: {feature_cols[:5]}{'...' if len(feature_cols) > 5 else ''}")
        
        # Initialize model
        model_hybrid = TCNTransformerHybrid(
            input_size=input_features,
            output_size=1,  # forecasting a single value per quantile
            horizon=config.FORECAST_HORIZON,
            num_quantiles=len(config.QUANTILES),
            tcn_channels=[32, 64],
            d_model=64,
            nhead=4,
            num_layers=2
        )
        
        print("✅ TCN+Transformer Hybrid model initialized successfully.")
        print(f"Model parameters: {sum(p.numel() for p in model_hybrid.parameters()):,}")
        
        # Store feature columns for later use
        model_feature_cols = feature_cols
        
    except Exception as e:
        pipeline_state.invalidate(f"Error initializing model: {str(e)}")
        model_hybrid = None
        model_feature_cols = []
else:
    model_hybrid = None
    model_feature_cols = []

--- TCN+Transformer Hybrid Model Definition ---
Input features: 15
Feature columns: ['hour', 'dayofweek', 'month', 'year', 'dayofyear']...


/home/panchani.d/.conda/envs/pytorch_env/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


✅ TCN+Transformer Hybrid model initialized successfully.
Model parameters: 572,516


# Training Utilities

### 🏋️ Training Time: Let's Get This Model Buff

*It's the moment of truth! We have our brainy model and our clean, feature-rich data. Now it's time to put them together and start the training process. This is where the model learns the patterns we've been preparing it for.*

**The Workout Plan**
This block contains all the machinery for our model's training regimen. Here's the play-by-play:

* **Creating Sequences:** First, we run our data through create_sequences. This function is like a cookie-cutter, chopping our long timeline into smaller, bite-sized "look back at this, predict that" samples that the model can actually learn from.

* **The Loss Function (Pinball Loss):** How does a model know if it's right or wrong? It uses a loss function. Since we're predicting a range of possible outcomes (quantiles) and not just one number, we use a special loss function called pinball_loss. It cleverly scores our model on how accurate its entire predicted range is, not just its median guess.

* **The Trainer (train_pytorch_model):** This is the head coach. It manages the whole workout, sending batches of data to the model, calculating the loss, and telling the model how to adjust its internal "weights" to get better. It runs this loop for several rounds (epochs).

* **The Spotter (EarlyStopper):** We also have a spotter to prevent over-training. The EarlyStopper watches the model's performance on a separate validation dataset. If the model stops improving after a few epochs, the spotter calls it a day, saving us from wasting time and ending up with a less effective model.

In [7]:
import time
from torch.utils.data import TensorDataset, DataLoader

def pinball_loss(y_pred, y_true, quantiles):
    """
    Calculates the pinball loss for quantile regression.
    y_pred shape: (batch_size, horizon, num_quantiles)
    y_true shape: (batch_size, horizon)
    """
    y_true = y_true.unsqueeze(-1)  # Add quantile dimension for broadcasting
    error = y_true - y_pred
    # Move quantiles to the correct device
    q_tensor = torch.tensor(quantiles, device=y_pred.device).view(1, 1, -1)
    loss = torch.max((q_tensor * error), ((q_tensor - 1) * error))
    return loss.mean()

class EarlyStopper:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

def create_sequences(data, lookback, horizon, feature_cols, target_col):
    """
    Creates sequences of features (X) and targets (y).
    This version explicitly uses the provided feature_cols list.
    """
    if data.empty or target_col not in data.columns:
        return np.array([], dtype=np.float32).reshape(0, lookback, len(feature_cols)), np.array([], dtype=np.float32).reshape(0, horizon)
    
    # Check if all feature columns exist
    missing_cols = [col for col in feature_cols if col not in data.columns]
    if missing_cols:
        print(f"Warning: Missing feature columns: {missing_cols}")
        feature_cols = [col for col in feature_cols if col in data.columns]
    
    X, y = [], []
    # Ensure there's enough data to create at least one sequence
    if len(data) < lookback + horizon:
        return np.array([], dtype=np.float32).reshape(0, lookback, len(feature_cols)), np.array([], dtype=np.float32).reshape(0, horizon)
        
    for i in range(len(data) - lookback - horizon + 1):
        # Select ONLY the specified feature columns for the input sequence
        X.append(data[feature_cols].iloc[i:(i + lookback)].values)
        y.append(data[target_col].iloc[(i + lookback):(i + lookback + horizon)].values)
        
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

def train_pytorch_model(model, train_df, val_df, config, feature_cols):
    """
    Trains a PyTorch model with proper error handling and validation.
    """
    if not check_pipeline_state("Model training"):
        return None, {}
    
    if model is None or train_df.empty:
        pipeline_state.invalidate("Cannot train: model is None or training data is empty")
        return None, {}
    
    try:
        print("\n--- Training Pure PyTorch TCN+Transformer Model ---")
        device = torch.device("cuda" if config.env_flags.get('CUDA_AVAILABLE', False) else "cpu")
        print(f"Using device: {device}")
        
        model.to(device)
        target_col = f"{config.TARGET_COLUMN}_scaled"
        
        # Create training sequences
        X_train, y_train = create_sequences(train_df, config.LOOKBACK_WINDOW, config.FORECAST_HORIZON, feature_cols, target_col)
        
        if X_train.shape[0] == 0:
            pipeline_state.invalidate("No training sequences could be created")
            return None, {}
        
        print(f"Training sequences: {X_train.shape[0]} samples")
        
        # Create validation sequences
        X_val, y_val = create_sequences(val_df, config.LOOKBACK_WINDOW, config.FORECAST_HORIZON, feature_cols, target_col)
        
        if X_val.shape[0] == 0:
            print("Warning: No validation sequences available - training without validation")
            use_validation = False
        else:
            print(f"Validation sequences: {X_val.shape[0]} samples")
            use_validation = True
        
        # Create data loaders
        train_loader = DataLoader(
            TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train)), 
            batch_size=config.BATCH_SIZE, 
            shuffle=True
        )
        
        if use_validation:
            val_loader = DataLoader(
                TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val)), 
                batch_size=config.BATCH_SIZE
            )
        
        # Setup training components
        optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE)
        early_stopper = EarlyStopper(patience=config.PATIENCE) if use_validation else None
        
        if use_validation:
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=config.PATIENCE // 2)
        
        scaler = torch.amp.GradScaler('cuda', enabled=config.USE_AMP) if torch.cuda.is_available() else torch.amp.GradScaler('cpu', enabled=False)
        progress_bar = get_progress_bar()
        
        train_losses = []
        val_losses = []
        
        # Training loop
        for epoch in range(config.EPOCHS):
            model.train()
            train_loss = 0
            epoch_start_time = time.time()
            
            for X_batch, y_batch in progress_bar(train_loader, desc=f"Epoch {epoch+1}/{config.EPOCHS} [Train]", leave=False):
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                
                with torch.amp.autocast('cuda' if torch.cuda.is_available() else 'cpu', enabled=config.USE_AMP):
                    outputs = model(X_batch)
                    loss = pinball_loss(outputs, y_batch, config.QUANTILES)
                
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                
                train_loss += loss.item()
            
            train_loss /= len(train_loader)
            train_losses.append(train_loss)
            
            # Validation
            val_loss = None
            if use_validation:
                model.eval()
                val_loss_total = 0
                with torch.no_grad():
                    for X_batch, y_batch in progress_bar(val_loader, desc=f"Epoch {epoch+1}/{config.EPOCHS} [Val]", leave=False):
                        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                        with torch.amp.autocast('cuda' if torch.cuda.is_available() else 'cpu', enabled=config.USE_AMP):
                            outputs = model(X_batch)
                            loss = pinball_loss(outputs, y_batch, config.QUANTILES)
                        val_loss_total += loss.item()
                
                val_loss = val_loss_total / len(val_loader)
                val_losses.append(val_loss)
            
            epoch_duration = time.time() - epoch_start_time
            
            val_loss_str = f"{val_loss:.4f}" if val_loss is not None else "N/A"
            print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss_str} | Time: {epoch_duration:.2f}s")
            
            # Early stopping and learning rate scheduling
            if use_validation and val_loss is not None:
                if early_stopper and early_stopper(val_loss):
                    print("Early stopping triggered.")
                    break
                scheduler.step(val_loss)
        
        pipeline_state.model_trained = True
        print("Training complete.")
        
        return model, {
            "train_losses": train_losses, 
            "val_losses": val_losses if use_validation else [],
            "X_val": X_val if use_validation else np.array([]),
            "y_val": y_val if use_validation else np.array([])
        }
        
    except Exception as e:
        pipeline_state.invalidate(f"Error during training: {str(e)}")
        return None, {}

print("Training utilities defined successfully.")

Training utilities defined successfully.


# Model Training Execution
### 💪 The Main Event: Kicking Off the Training

*Alright, the warm-up is over. We've prepped the data, built the brain, and laid out the workout plan. This cell is the starting whistle.*

* It hands our data and the model to the train_pytorch_model coach and kicks off the training loop. You'll see the progress for each round (epoch) as the model learns from the data and gets progressively smarter.

* We don't even let the model catch its breath. As soon as the training is finished, we immediately have it make predictions on the validation data. This gives us our first, honest look at how well it actually learned its lesson.

In [8]:
# Execute model training
if check_pipeline_state("Training execution") and model_hybrid is not None and train_indices is not None:
    try:
        # Prepare training and validation data
        train_data = featured_df.iloc[train_indices]
        val_data = featured_df.iloc[test_indices]
        
        print(f"Training data shape: {train_data.shape}")
        print(f"Validation data shape: {val_data.shape}")
        
        # Train the model
        trained_model, training_history = train_pytorch_model(
            model_hybrid, 
            train_data, 
            val_data, 
            config, 
            model_feature_cols
        )
        
        if trained_model is not None and pipeline_state.model_trained:
            print("Model training completed successfully.")
            
            # Store training artifacts
            X_val = training_history.get('X_val', np.array([]))
            y_val = training_history.get('y_val', np.array([]))
            
            if X_val.shape[0] > 0:
                print(f"Validation sequences available: {X_val.shape}")
                
                # Generate predictions
                device = torch.device("cuda" if config.env_flags.get('CUDA_AVAILABLE', False) else "cpu")
                trained_model.to(device)
                trained_model.eval()
                
                with torch.no_grad():
                    X_val_tensor = torch.from_numpy(X_val).to(device)
                    y_pred_quantiles = trained_model(X_val_tensor).cpu().numpy()
                
                pipeline_state.predictions_generated = True
                print(f"Predictions generated: {y_pred_quantiles.shape}")
            else:
                print("No validation data available for predictions.")
                y_pred_quantiles = np.array([])
        else:
            pipeline_state.invalidate("Model training failed")
            trained_model = None
            X_val, y_val, y_pred_quantiles = np.array([]), np.array([]), np.array([])
            
    except Exception as e:
        pipeline_state.invalidate(f"Error during training execution: {str(e)}")
        trained_model = None
        X_val, y_val, y_pred_quantiles = np.array([]), np.array([]), np.array([])
else:
    print("Skipping model training - prerequisites not met.")
    trained_model = None
    X_val, y_val, y_pred_quantiles = np.array([]), np.array([]), np.array([])

Training data shape: (720, 17)
Validation data shape: (12, 17)

--- Training Pure PyTorch TCN+Transformer Model ---
Using device: cuda
Training sequences: 637 samples


/tmp/ipykernel_2533330/1220099311.py:103: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Epoch 1/10 [Train]:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 01 | Train Loss: 0.5014 | Val Loss: N/A | Time: 3.62s


Epoch 2/10 [Train]:   0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch 08 | Train Loss: 0.3053 | Val Loss: N/A | Time: 0.08s


Epoch 9/10 [Train]:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 09 | Train Loss: 0.2997 | Val Loss: N/A | Time: 0.09s


Epoch 10/10 [Train]:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 10 | Train Loss: 0.2929 | Val Loss: N/A | Time: 0.09s
Training complete.
Model training completed successfully.
No validation data available for predictions.


# Evaluation & Metrics
### 🏆 The Report Card: Did We Pass?

Training is done and our model has made its first predictions. But are they any good? It's time to grade our model's performance.

This final block is the judge and jury. It takes the model's predictions and compares them to the actual, real-world answers. Before it can deliver a fair verdict, it has to reverse the scaling we applied way back at the beginning, translating the model's scaled-down numbers back into their original, real-world units.

**The Final Scores**
We're using a few key metrics to get a well-rounded view of our model's performance:

* MAE & RMSE: These are the classics. They tell us, on average, how far off our model's main prediction was from the actual value. Lower is better.

* MAPE: This metric gives us the error as a percentage, which is often easier to understand. A MAPE of 10% means we were off by 10% on average.

* Pinball Loss: This is the official score for our quantile predictions. It grades the entire predicted range, not just the single best guess.

* Coverage: This checks if the real answer fell within our predicted range (between the P10 and P90 quantiles). A good model should have its coverage close to the 80% range it was aiming for.

In [9]:
import numpy as np
import pandas as pd

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero
    mask = y_true != 0
    if not np.any(mask):
        return np.inf
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def coverage(y_true, y_lower, y_upper):
    return np.mean((y_true >= y_lower) & (y_true <= y_upper)) * 100

def pinball_loss_metric(y_true, y_pred_quantiles, quantiles):
    """Numpy version of the pinball loss for evaluation."""
    if y_pred_quantiles.size == 0 or y_true.size == 0:
        return np.inf
        
    loss = 0
    y_true_exp = np.expand_dims(y_true, axis=-1)
    error = y_true_exp - y_pred_quantiles
    for i, q in enumerate(quantiles):
        loss += np.mean(np.maximum(q * error[..., i], (q - 1) * error[..., i]))
    return loss / len(quantiles)

def evaluate_forecast(y_true, y_pred_quantiles, quantiles, scaler, target_col):
    """
    Computes a dictionary of evaluation metrics.
    Assumes y_pred_quantiles and y_true are scaled.
    """
    if not check_pipeline_state("Forecast evaluation"):
        return {}
    
    if y_pred_quantiles.size == 0 or y_true.size == 0:
        print("Warning: Empty predictions or targets, cannot evaluate")
        return {}
    
    try:
        # Find quantile indices
        p10_idx = quantiles.index(0.1) if 0.1 in quantiles else 0
        p50_idx = quantiles.index(0.5) if 0.5 in quantiles else len(quantiles)//2
        p90_idx = quantiles.index(0.9) if 0.9 in quantiles else -1
        
        # Inverse transform to original scale
        if isinstance(scaler, dict):  # Manual scaler
            mean, std = scaler['mean'], scaler['std']
            y_true_orig = y_true * std + mean
            y_pred_quantiles_orig = y_pred_quantiles * std + mean
        else:  # Scikit-learn scaler
            y_true_orig = scaler.inverse_transform(y_true.reshape(-1, 1)).flatten()
            
            # Reshape predictions for inverse transform
            num_samples, horizon, num_quantiles = y_pred_quantiles.shape
            preds_flat = y_pred_quantiles.reshape(-1, num_quantiles)
            preds_orig_flat = np.zeros_like(preds_flat)
            for i in range(num_quantiles):
                preds_orig_flat[:, i] = scaler.inverse_transform(preds_flat[:, i].reshape(-1, 1)).flatten()
            y_pred_quantiles_orig = preds_orig_flat.reshape(num_samples, horizon, num_quantiles)
        
        y_pred_p50_orig = y_pred_quantiles_orig[..., p50_idx].flatten()
        y_true_flat_orig = y_true_orig.flatten()
        
        metrics = {
            "MAE": mae(y_true_flat_orig, y_pred_p50_orig),
            "RMSE": rmse(y_true_flat_orig, y_pred_p50_orig),
            "MAPE": mape(y_true_flat_orig, y_pred_p50_orig),
            "Pinball Loss": pinball_loss_metric(y_true_orig, y_pred_quantiles_orig, quantiles),
        }
        
        # Add coverage if we have the right quantiles
        if 0.1 in quantiles and 0.9 in quantiles:
            metrics["Coverage (P10-P90)"] = coverage(
                y_true_orig, 
                y_pred_quantiles_orig[..., p10_idx], 
                y_pred_quantiles_orig[..., p90_idx]
            )
        
        return metrics
        
    except Exception as e:
        print(f"Error during evaluation: {str(e)}")
        return {}

# Execute evaluation if we have valid predictions
if (check_pipeline_state("Evaluation execution") and 
    pipeline_state.predictions_generated and 
    'y_pred_quantiles' in locals() and 
    y_pred_quantiles.size > 0 and 
    'y_val' in locals() and 
    y_val.size > 0):
    
    try:
        print("\n--- Evaluation Metrics ---")
        
        metrics = evaluate_forecast(y_val, y_pred_quantiles, config.QUANTILES, scaler, config.TARGET_COLUMN)
        
        if metrics:
            for key, value in metrics.items():
                if np.isfinite(value):
                    print(f"{key}: {value:.4f}")
                else:
                    print(f"{key}: Invalid (inf/nan)")
        else:
            print("No valid metrics could be calculated")
            
    except Exception as e:
        print(f"Error during evaluation execution: {str(e)}")
        metrics = {}
else:
    print("Skipping evaluation - no valid predictions available")
    metrics = {}

print("Evaluation metrics functions defined successfully.")

Skipping evaluation - no valid predictions available
Evaluation metrics functions defined successfully.


# What-If Scenario Engine

### 🔮 The Crystal Ball: Playing with the Future

*A forecast is just a starting point. The real fun begins when we ask, "What if...?"*

This final block is our what-if engine, a playground for stress-testing our forecast. We're about to throw a wrench in the works to see how our calm, predictable energy grid handles a sudden, massive change.

**Here's the Game Plan:**
* **Unleash the EV Tsunami:** First, we simulate a sudden rush of thousands of Electric Vehicles all plugging in after work. The generate_ev_load_profile function creates this tidal wave of new energy demand. You are in control of the chaos, using the UI sliders to decide just how many EVs show up to the party.

* **The Aftermath:** We then smash this new wave of demand onto our original, "business-as-usual" forecast. This shows us the new, combined reality.

The result is a clear picture and a summary of our little experiment, showing exactly how much stress this EV-pocalypse puts on the grid and what our new, much higher, peak demand looks like.

In [10]:
import numpy as np
import pandas as pd

if config.env_flags.get('USE_MATPLOTLIB', False):
    import matplotlib.pyplot as plt

def generate_ev_load_profile(
    num_evs=1000, 
    charging_power_kw=7.2, 
    charging_duration_h=5, 
    start_time_mean=18, 
    start_time_std=2, 
    horizon_h=24,
    baseline_index=None
):
    """
    Generates a synthetic EV charging load profile for a 24-hour period.
    Each EV starts charging at a normally distributed start hour,
    charges at charging_power_kw for charging_duration_h hours.
    """
    if not check_pipeline_state("EV load profile generation"):
        return pd.Series()
        
    try:
        print(f"\n--- Generating EV scenario for {num_evs} EVs ---")
        total_load = np.zeros(horizon_h)
        start_times = np.random.normal(start_time_mean, start_time_std, num_evs)
        
        for start_hour in start_times:
            start_idx = int(round(start_hour)) % horizon_h
            end_idx = start_idx + int(round(charging_duration_h))
            # Add load for the charging duration, wrap around if needed
            for i in range(start_idx, end_idx):
                total_load[i % horizon_h] += charging_power_kw
        
        # Build index
        if baseline_index is not None:
            index = baseline_index[:horizon_h]
        else:
            index = pd.date_range('2024-01-01 00:00:00', periods=horizon_h, freq='H')
                
        ev_load_profile = pd.Series(total_load, index=index, name="ev_load_kw")
        print(f"Total EV energy generated: {total_load.sum():.1f} kWh "
              f"({num_evs} EVs × {charging_duration_h}h × {charging_power_kw} kW avg)")
        return ev_load_profile
        
    except Exception as e:
        print(f"❌ Error generating EV load profile: {str(e)}")
        return pd.Series()

def create_scenario_dataframe(baseline_forecast, baseline_index, ev_profile):
    """
    Creates a comprehensive scenario dataframe with baseline and EV loads.
    """
    try:
        scenario_df = pd.DataFrame({'baseline_load': baseline_forecast}, index=baseline_index)
        
        if len(ev_profile) > 0:
            aligned_ev_load = pd.Series(0.0, index=scenario_df.index)
            min_length = min(len(scenario_df), len(ev_profile))
            aligned_ev_load.iloc[:min_length] = ev_profile.iloc[:min_length].values
            scenario_df['ev_load'] = aligned_ev_load
        else:
            scenario_df['ev_load'] = 0.0
            
        scenario_df['total_load_with_ev'] = scenario_df['baseline_load'] + scenario_df['ev_load']
        
        return scenario_df
        
    except Exception as e:
        print(f"❌ Error creating scenario dataframe: {str(e)}")
        return pd.DataFrame()

# === Execute scenario generation ===
if (check_pipeline_state("Scenario generation") and 
    pipeline_state.predictions_generated and 
    'y_pred_quantiles' in locals() and 
    y_pred_quantiles.size > 0 and 
    'test_indices' in locals() and 
    test_indices is not None):
    
    try:
        print("\n--- What-If Scenario Analysis ---")
        
        # Baseline forecast from predictions
        p50_idx = config.QUANTILES.index(0.5) if 0.5 in config.QUANTILES else len(config.QUANTILES)//2
        baseline_forecast_scaled = y_pred_quantiles[0, :, p50_idx]
        
        if isinstance(scaler, dict):
            baseline_forecast = baseline_forecast_scaled * scaler['std'] + scaler['mean']
        else:
            baseline_forecast = scaler.inverse_transform(baseline_forecast_scaled.reshape(-1, 1)).flatten()
        
        # Forecast index
        val_data = featured_df.iloc[test_indices]
        forecast_start_idx = config.LOOKBACK_WINDOW
        forecast_end_idx = forecast_start_idx + config.FORECAST_HORIZON
        
        if len(val_data) >= forecast_end_idx:
            forecast_index = val_data.index[forecast_start_idx:forecast_end_idx]
        else:
            start_time = val_data.index[0] if len(val_data) > 0 else pd.Timestamp('2024-01-01')
            forecast_index = pd.date_range(start_time, periods=config.FORECAST_HORIZON, freq='H')
        
        print(f"Baseline forecast range: {forecast_index[0]} → {forecast_index[-1]}")
        
        # === 🔑 Generate EV load profile using UI sliders ===
        ev_load = generate_ev_load_profile(
            num_evs=num_evs_slider.value,
            charging_power_kw=charging_power_slider.value,
            charging_duration_h=charging_duration_slider.value,
            horizon_h=config.FORECAST_HORIZON,
            baseline_index=forecast_index
        )
        
        # Create scenario dataframe
        scenario_df = create_scenario_dataframe(baseline_forecast, forecast_index, ev_load)
        
        if not scenario_df.empty:
            print(f"✅ Scenario created successfully with {len(scenario_df)} time points")
            print("\nScenario Summary:")
            print(f"  Baseline peak load: {scenario_df['baseline_load'].max():.1f} kW")
            print(f"  Total EV load: {scenario_df['ev_load'].sum():.1f} kWh")
            print(f"  Peak load with EVs: {scenario_df['total_load_with_ev'].max():.1f} kW")
            print(f"  Peak increase: {((scenario_df['total_load_with_ev'].max() / scenario_df['baseline_load'].max() - 1) * 100):.1f}%")
            
            if config.env_flags.get('USE_MATPLOTLIB', False):
                plt.figure(figsize=(12, 6))
                scenario_df[['baseline_load', 'total_load_with_ev']].plot()
                plt.title("What-If Scenario: Baseline Load vs. Load with Uncontrolled EV Charging")
                plt.ylabel("Load (kW)")
                plt.grid(True, which='both', linestyle='--', linewidth=0.5)
                plt.legend(['Baseline Load', 'Load with EVs'])
                plt.tight_layout()
                plt.show()
        else:
            print("❌ Failed to create scenario dataframe")
            
    except Exception as e:
        print(f"❌ Error during scenario generation: {str(e)}")
        scenario_df = pd.DataFrame()
else:
    print("Skipping What-If Scenario Engine - prerequisites not met")
    scenario_df = pd.DataFrame()

print("What-If Scenario Engine completed.")

Skipping What-If Scenario Engine - prerequisites not met
What-If Scenario Engine completed.


# Load Shifting Optimizer
### 🧠 Smart Charging: Playing Tetris with Energy

*Okay, our last "what-if" scenario created a huge, chaotic spike in energy demand. That's bad for the grid and expensive for everyone. But what if we could be smarter about it? What if we could tell all those EVs to charge when energy is cheapest and most plentiful?*

This block is our smart-charging optimizer. Think of it as playing a game of Tetris with the EV charging load. It takes that big, awkward block of energy demand and finds the perfect empty spots to drop it into, flattening the overall demand curve.

**The Strategy: A Greedy Approach**
*Our optimizer, greedy_load_shifter, uses a simple but powerful "greedy" strategy:*

* **Identify the Quiet Hours:** First, it looks for the off-peak window (e.g., between 10 PM and 7 AM) when the baseline energy demand is lowest.

* **Find the Best Spots:** Within that window, it finds the absolute quietest hours—the bottom of the "valley" in our energy usage graph.

* **Shift the Load:** It then takes the total energy needed by the EVs and strategically pours it into those quietest hours, starting with the very lowest and filling it up.

The result is a much smoother, flatter energy profile. Instead of a scary, sharp peak, the demand is spread out, which is healthier for the grid and cheaper for consumers. The final plot shows the chaotic "uncontrolled" spike versus our new, much calmer "optimized" load.

In [11]:
import numpy as np
import pandas as pd

if config.env_flags.get('USE_MATPLOTLIB', False):
    import matplotlib.pyplot as plt

def greedy_load_shifter(baseline_load, flexible_load, charging_window=(22, 7), charging_duration_h=None):
    """
    Greedy optimizer:
      - Shifts flexible EV load into lowest-load off-peak hours.
      - Respects user-defined charging window and charging duration.
    Returns optimized total load and shifted flexible load.
    """
    if not check_pipeline_state("Load shifting optimization"):
        return pd.Series(), pd.Series()
        
    if baseline_load.empty or flexible_load.empty:
        print("⚠️ Warning: Empty load profiles provided to optimizer")
        return baseline_load.copy(), flexible_load.copy()
    
    try:
        print("\n--- Running Greedy Load Shifting Optimizer ---")
        
        optimized_flexible_load = pd.Series(0.0, index=flexible_load.index)
        total_energy_to_shift = flexible_load.sum()  # Assuming hourly data, sum ~ kWh
        
        print(f"Total EV energy to shift: {total_energy_to_shift:.1f} kWh")
        
        # Identify valid charging slots
        start, end = charging_window
        if start > end:  # Overnight window like 22 → 7
            valid_hours = (baseline_load.index.hour >= start) | (baseline_load.index.hour < end)
        else:  # Daytime window
            valid_hours = (baseline_load.index.hour >= start) & (baseline_load.index.hour < end)
        
        available_slots = baseline_load[valid_hours]
        print(f"Available off-peak slots: {len(available_slots)} hours")
        
        if len(available_slots) > 0:
            # Sort slots by baseline load (lowest first)
            available_slots_sorted = available_slots.sort_values()
            
            # If user specified duration, restrict number of slots
            if charging_duration_h is not None:
                slots_needed = min(int(charging_duration_h), len(available_slots_sorted))
                selected_slots = available_slots_sorted.index[:slots_needed]
            else:
                selected_slots = available_slots_sorted.index  # use all
            
            print(f"Using {len(selected_slots)} slots for charging")
            
            # True greedy allocation: fill lowest slots until energy is assigned
            total_energy_remaining = total_energy_to_shift
            energy_per_slot = total_energy_to_shift / len(selected_slots)
            
            for ts in selected_slots:
                if total_energy_remaining <= 0:
                    break
                allocation = min(energy_per_slot, total_energy_remaining)
                optimized_flexible_load[ts] = allocation
                total_energy_remaining -= allocation
            
            if total_energy_remaining > 0:
                print(f"⚠️ {total_energy_remaining:.1f} kWh could not be shifted (not enough slots)")
            
            print(f"✅ Energy distributed across {len(selected_slots)} slots")
        else:
            print("⚠️ No available off-peak slots to shift load into. Load remains unchanged.")
            optimized_flexible_load = flexible_load.copy()

        optimized_total_load = baseline_load + optimized_flexible_load

        # Metrics
        original_peak = (baseline_load + flexible_load).max()
        optimized_peak = optimized_total_load.max()
        peak_reduction = ((original_peak - optimized_peak) / original_peak * 100) if original_peak > 0 else 0
        
        original_lf = (baseline_load + flexible_load).mean() / original_peak if original_peak > 0 else 0
        optimized_lf = optimized_total_load.mean() / optimized_peak if optimized_peak > 0 else 0
        lf_improvement = (optimized_lf - original_lf) * 100
        
        print("Optimization Results:")
        print(f"  - Original Peak Load: {original_peak:,.2f} kW")
        print(f"  - Optimized Peak Load: {optimized_peak:,.2f} kW")
        print(f"  - Peak Reduction: {peak_reduction:.2f}%")
        print(f"  - Load Factor Improvement: {lf_improvement:.2f} percentage points")

        return optimized_total_load, optimized_flexible_load
        
    except Exception as e:
        print(f"❌ Error during load shifting optimization: {str(e)}")
        return baseline_load.copy(), flexible_load.copy()

# Execute optimization if scenario is available
if (check_pipeline_state("Optimization execution") and 
    'scenario_data' in locals() and 
    not scenario_data.empty and 
    'baseline_load' in scenario_data.columns and 
    'ev_load' in scenario_data.columns):
    
    try:
        print("\n--- Load Shifting Optimization ---")
        
        # Pass charging_duration_h from UI if available
        charging_duration_h = charging_duration_slider.value if 'charging_duration_slider' in locals() else None
        
        optimized_load, shifted_ev_load = greedy_load_shifter(
            scenario_data['baseline_load'], 
            scenario_data['ev_load'],
            charging_window=(22, 7),
            charging_duration_h=charging_duration_h
        )
        
        if not optimized_load.empty and not shifted_ev_load.empty:
            scenario_data['optimized_total_load'] = optimized_load
            scenario_data['shifted_ev_load'] = shifted_ev_load
            
            if config.env_flags.get('USE_MATPLOTLIB', False):
                plt.figure(figsize=(12, 6))
                scenario_data[['total_load_with_ev', 'optimized_total_load']].plot(style=['--', '-'])
                scenario_data['baseline_load'].plot(style=':', color='gray')
                plt.title("Optimization Result: Uncontrolled vs. Optimized Charging")
                plt.ylabel("Load (kW)")
                plt.legend(['Uncontrolled Total Load', 'Optimized Total Load', 'Baseline Load'])
                plt.grid(True, which='both', linestyle='--', linewidth=0.5)
                plt.tight_layout()
                plt.show()
                
            print("✅ Optimization completed successfully.")
        else:
            print("❌ Optimization failed to produce valid results.")
            
    except Exception as e:
        print(f"❌ Error during optimization execution: {str(e)}")
else:
    print("Skipping Load Shifting Optimizer - no valid scenario available")

print("Load Shifting Optimizer completed.")

Skipping Load Shifting Optimizer - no valid scenario available
Load Shifting Optimizer completed.


# Advanced Reporting Dashboard
### 📊 The Final Verdict: The Executive Dashboard

*We've run our simulation and optimized the chaos. Now it's time for the final reveal. This block is the grand finale, where we crunch all the numbers and present them in a clear, comprehensive dashboard. Think of it as the executive summary that tells us whether our smart-charging strategy was a genius move or a waste of time.*

**What's on the Report?** \
**The generate_advanced_report function is our in-house data artist and accountant. It takes all our scenario data and calculates the most important metrics to answer two key questions: "How much stress did we take off the grid?" and "How much money did we save?"**

* **Grid Impact Analysis:** This section focuses on the health of the power grid. We're looking at the Peak Reduction (how much we squashed that scary spike) and the Load Factor Improvement. A better load factor means we're using our power infrastructure more efficiently, which is a big win for everyone.

* **Economic Impact:** The calculate_cost_savings function plays accountant, applying different prices for peak (expensive) and off-peak (cheap) hours. It calculates the cost of the chaotic, uncontrolled charging versus our smart, optimized charging, and presents the final cost savings as a simple percentage.

The end result is a beautiful dashboard with charts and a summary report, giving us a complete, at-a-glance understanding of just how effective our optimization strategy really was.

In [12]:
import numpy as np
import pandas as pd

if config.env_flags.get('USE_MATPLOTLIB', False):
    import matplotlib.pyplot as plt

def calculate_cost_savings(scenario_df, peak_price=0.30, off_peak_price=0.10, peak_hours=(16, 21)):
    """
    Calculates cost savings from load shifting based on time-of-use pricing.
    """
    try:
        if 'ev_load' not in scenario_df.columns or 'shifted_ev_load' not in scenario_df.columns:
            return 0, 0, 0
            
        # Create price series
        price_series = pd.Series(index=scenario_df.index, dtype=float)
        start_hour, end_hour = peak_hours
        
        for timestamp in scenario_df.index:
            hour = timestamp.hour
            if start_hour <= hour <= end_hour:
                price_series[timestamp] = peak_price
            else:
                price_series[timestamp] = off_peak_price
        
        # Calculate costs
        cost_before = (scenario_df['ev_load'] * price_series).sum()
        cost_after = (scenario_df['shifted_ev_load'] * price_series).sum()
        cost_savings = cost_before - cost_after
        cost_savings_percent = (cost_savings / cost_before * 100) if cost_before > 0 else 0
        
        return cost_before, cost_after, cost_savings_percent
        
    except Exception as e:
        print(f"Error calculating cost savings: {str(e)}")
        return 0, 0, 0

def generate_advanced_report(scenario_df):
    """
    Calculates advanced grid metrics and generates a comprehensive dashboard.
    """
    if not check_pipeline_state("Advanced reporting"):
        return
        
    if scenario_df.empty:
        print("Cannot generate report because scenario_df is empty.")
        return

    try:
        print("\n--- Generating Advanced Report ---")
        
        # Extract load profiles
        uncontrolled_load = scenario_df['total_load_with_ev']
        baseline_load = scenario_df['baseline_load']
        
        # Check if optimization was performed
        if 'optimized_total_load' in scenario_df.columns:
            optimized_load = scenario_df['optimized_total_load']
            optimization_performed = True
        else:
            optimized_load = uncontrolled_load.copy()
            optimization_performed = False
            
        # Calculate key metrics
        total_energy_shifted_mwh = scenario_df['ev_load'].sum() / 1000  # Convert kW to MWh
        
        # Load Factor calculations
        lf_baseline = baseline_load.mean() / baseline_load.max() if baseline_load.max() > 0 else 0
        lf_uncontrolled = uncontrolled_load.mean() / uncontrolled_load.max() if uncontrolled_load.max() > 0 else 0
        lf_optimized = optimized_load.mean() / optimized_load.max() if optimized_load.max() > 0 else 0
        lf_improvement = (lf_optimized - lf_uncontrolled) * 100
        
        # Peak reduction
        peak_reduction_percent = ((uncontrolled_load.max() - optimized_load.max()) / uncontrolled_load.max() * 100) if uncontrolled_load.max() > 0 else 0
        
        # Cost analysis
        cost_before, cost_after, cost_savings_percent = calculate_cost_savings(scenario_df)
        
        # Print summary metrics
        print("Grid Impact Analysis:")
        print(f"  Total EV Energy: {total_energy_shifted_mwh:.2f} MWh")
        print(f"  Baseline Peak: {baseline_load.max():.1f} kW")
        print(f"  Uncontrolled Peak: {uncontrolled_load.max():.1f} kW")
        print(f"  Optimized Peak: {optimized_load.max():.1f} kW")
        print(f"  Peak Reduction: {peak_reduction_percent:.1f}%")
        print(f"  Load Factor Improvement: {lf_improvement:.2f} percentage points")
        print(f"  Cost Savings: {cost_savings_percent:.1f}%")
        
        # Generate visualization dashboard if matplotlib is available
        if config.env_flags.get('USE_MATPLOTLIB', False):
            fig, axs = plt.subplots(2, 2, figsize=(16, 12))
            fig.suptitle('Smart Grid Optimization Dashboard', fontsize=20, fontweight='bold')
            
            # Plot 1: Load Profiles (Time Series)
            ax = axs[0, 0]
            baseline_load.plot(ax=ax, style=':', color='gray', label='Baseline (no EVs)', alpha=0.8)
            uncontrolled_load.plot(ax=ax, style='--', color='red', label='With Uncontrolled EVs', alpha=0.8)
            if optimization_performed:
                optimized_load.plot(ax=ax, style='-', color='green', label='With Optimized EVs', alpha=0.9, linewidth=2)
            ax.set_title('Load Profile Comparison', fontsize=14, fontweight='bold')
            ax.set_ylabel('Electricity Demand (kW)')
            ax.grid(True, alpha=0.3)
            ax.legend()
            
            # Plot 2: Load Distribution (Histogram)
            ax = axs[0, 1]
            uncontrolled_load.plot(kind='hist', bins=20, ax=ax, alpha=0.7, color='red', label='Uncontrolled')
            if optimization_performed:
                optimized_load.plot(kind='hist', bins=20, ax=ax, alpha=0.7, color='green', label='Optimized')
            ax.axvline(uncontrolled_load.max(), color='red', linestyle='--', alpha=0.8, 
                      label=f'Uncontrolled Peak: {uncontrolled_load.max():.0f} kW')
            if optimization_performed:
                ax.axvline(optimized_load.max(), color='green', linestyle='--', alpha=0.8, 
                          label=f'Optimized Peak: {optimized_load.max():.0f} kW')
            ax.set_title('Load Distribution', fontsize=14, fontweight='bold')
            ax.set_xlabel('Electricity Demand (kW)')
            ax.set_ylabel('Frequency')
            ax.legend()
            
            # Plot 3: EV Charging Patterns
            ax = axs[1, 0]
            if 'ev_load' in scenario_df.columns:
                scenario_df['ev_load'].plot(ax=ax, style='--', color='orange', label='Original EV Load', alpha=0.8)
            if 'shifted_ev_load' in scenario_df.columns and optimization_performed:
                scenario_df['shifted_ev_load'].plot(ax=ax, style='-', color='blue', label='Shifted EV Load', linewidth=2)
            ax.set_title('EV Charging Patterns', fontsize=14, fontweight='bold')
            ax.set_ylabel('EV Load (kW)')
            ax.set_xlabel('Time')
            ax.grid(True, alpha=0.3)
            ax.legend()
            
            # Plot 4: Summary Metrics
            ax = axs[1, 1]
            ax.axis('off')
            
            summary_text = (
                "OPTIMIZATION SUMMARY\n"
                "=" * 40 + "\n\n"
                f"Total EV Energy: {total_energy_shifted_mwh:.2f} MWh\n\n"
                f"Peak Load Reduction:\n"
                f"  {uncontrolled_load.max():,.0f} → {optimized_load.max():,.0f} kW\n"
                f"  Reduction: {peak_reduction_percent:.1f}%\n\n"
                f"Load Factor:\n"
                f"  Baseline: {lf_baseline:.3f}\n"
                f"  Uncontrolled: {lf_uncontrolled:.3f}\n"
                f"  Optimized: {lf_optimized:.3f}\n"
                f"  Improvement: +{lf_improvement:.1f} pts\n\n"
                f"Economic Impact:\n"
                f"  Original Cost: ${cost_before:.2f}\n"
                f"  Optimized Cost: ${cost_after:.2f}\n"
                f"  Savings: {cost_savings_percent:.1f}%\n\n"
                f"Grid Stress Reduction:\n"
                f"  Peak-to-Average Ratio: {optimized_load.max()/optimized_load.mean():.2f}\n"
                f"  Load Variability: {optimized_load.std()/optimized_load.mean():.3f}"
            )
            
            ax.text(0.05, 0.95, summary_text, ha='left', va='top', fontsize=11, 
                   fontfamily='monospace', transform=ax.transAxes,
                   bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.8))
            
            plt.tight_layout()
            plt.subplots_adjust(top=0.94)  # Make room for main title
            plt.show()
        
        return {
            'total_energy_mwh': total_energy_shifted_mwh,
            'peak_reduction_percent': peak_reduction_percent,
            'load_factor_improvement': lf_improvement,
            'cost_savings_percent': cost_savings_percent,
            'baseline_peak_kw': baseline_load.max(),
            'uncontrolled_peak_kw': uncontrolled_load.max(),
            'optimized_peak_kw': optimized_load.max()
        }
        
    except Exception as e:
        print(f"Error generating advanced report: {str(e)}")
        return {}

# Generate advanced report if scenario is available
if (check_pipeline_state("Advanced report generation") and 
    'scenario_df' in locals() and 
    not scenario_df.empty):
    
    try:
        report_metrics = generate_advanced_report(scenario_df)
        
        if report_metrics:
            print("\nAdvanced report generated successfully.")
        else:
            print("Advanced report generation failed.")
            
    except Exception as e:
        print(f"Error during advanced report generation: {str(e)}")
else:
    print("Skipping Advanced Reporting - no valid scenario available")
    report_metrics = {}

print("Advanced Reporting Dashboard completed.")

Skipping Advanced Reporting - no valid scenario available
Advanced Reporting Dashboard completed.


# 🩹 The Duct Tape Fix: A Bigger Validation Set

**So... it looks like our fancy time-traveling walk-forward split was a bit too precise. Sometimes it leaves us with a validation set so small that our model can't even get a proper workout on it. It's like trying to test-drive a car by only moving it one inch.**

This little function is our emergency duct tape.

It throws elegance out the window and does one simple thing: it carves off a big, guaranteed chunk of our data (say, the last 30%) to create a fat validation set. It's not as clever as our previous method, but it ensures we have more than enough data to get a reliable score and prevent our training pipeline from complaining.

Sometimes, you just need a bigger hammer. This is that hammer.

In [13]:
# Emergency fix for validation data
def create_larger_validation_split(data, train_ratio=0.8, lookback=48, horizon=12):
    """Create a simple train/test split with enough validation data"""
    total_samples = len(data)
    split_point = int(total_samples * train_ratio)
    
    train_indices = np.arange(0, split_point)
    val_indices = np.arange(split_point, total_samples)
    
    print(f"Split created: {len(train_indices)} train, {len(val_indices)} val samples")
    
    # Ensure validation has enough data
    min_needed = lookback + horizon
    if len(val_indices) >= min_needed:
        return [(train_indices, val_indices)]
    else:
        print(f"Warning: Validation needs {min_needed} samples, only has {len(val_indices)}")
        return None

# Test the fix
if 'featured_df' in globals() and not featured_df.empty:
    test_splits = create_larger_validation_split(featured_df, train_ratio=0.7)
    if test_splits:
        train_idx, val_idx = test_splits[0]
        print(f"Fixed split: Train {len(train_idx)}, Val {len(val_idx)} samples")

Split created: 73500 train, 31501 val samples
Fixed split: Train 73500, Val 31501 samples


# Interactive User Interface

### 🎮 The Command Center: You're in Control Now

*We've built all the individual pieces of our smart grid pipeline: the forecaster, the scenario generator, the optimizer, and the reporter. Now, it's time to put you in the driver's seat.*

This final, glorious block uses ipywidgets to wrap our entire project in a sleek, interactive dashboard. It's no longer just a script; it's a tool you can play with.

**How It Works**
* **The Control Panel:** You'll see a set of sliders and dropdowns. These are your levers of power. Want to see what 10,000 EVs would do? Drag the slider. Want to change the off-peak charging window? Tweak the hours.

* **The Big Green Button:** When you're ready, hit 'Run Full Pipeline'. This one click triggers the entire end-to-end process: it loads the data, trains the model, creates your custom EV apocalypse, optimizes it, and generates the final report based on your exact settings.

* **The Main Screen:** All the output—from the training logs to the final dashboard charts—will appear right below the button. You can run new experiments as many times as you like without ever having to touch the code again.

Go ahead, play the role of a grid operator and see if you can find the optimal strategy!

In [14]:
if config.env_flags.get('USE_IPYWIDGETS', False):
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    import torch
    import numpy as np
    import pandas as pd

    def validate_ui_inputs(ev_count, charging_power, charging_duration, start_hour, end_hour):
        warnings = []
        if ev_count <= 0:
            warnings.append("Number of EVs should be > 0")
        if charging_power <= 0:
            warnings.append("Charging power should be > 0 kW")
        if charging_duration <= 0 or charging_duration > 24:
            warnings.append("Charging duration must be between 1 and 24 hours")
        if start_hour == end_hour:
            warnings.append("Off-peak start and end hour are the same, window invalid")
        return warnings

    def create_interactive_interface():
        """Creates an interactive interface for running the pipeline with different parameters."""
        
        if not check_pipeline_state("Interactive interface creation"):
            return
        
        try:
            # Widget definitions
            dataset_selector = widgets.Dropdown(
                options=[('OPSD Germany', 'opsd'), ('UCI Household', 'uci_household')],
                value='opsd',
                description='Dataset:',
                style={'description_width': 'initial'}
            )
            
            ev_slider = widgets.IntSlider(value=1000, min=0, max=10000, step=500,
                                          description='Number of EVs:', style={'description_width': 'initial'})
            
            charging_power_slider = widgets.FloatSlider(value=7.2, min=3.0, max=22.0, step=0.1,
                                                        description='Charging Power (kW):', style={'description_width': 'initial'})
            
            charging_duration_slider = widgets.IntSlider(value=5, min=1, max=24, step=1,
                                                         description='Charging duration (h):', style={'description_width': 'initial'})
            
            charging_window_start = widgets.IntSlider(value=22, min=0, max=23,
                                                      description='Off-peak Start (hour):', style={'description_width': 'initial'})
            
            charging_window_end = widgets.IntSlider(value=7, min=0, max=23,
                                                    description='Off-peak End (hour):', style={'description_width': 'initial'})
            
            run_button = widgets.Button(description='Run Full Pipeline', button_style='success',
                                        tooltip='Train model, generate scenarios, and optimize', icon='play')
            
            output_area = widgets.Output()
            
            summary_panel = widgets.HTML()
            
            def on_run_button_clicked(b):
                with output_area:
                    clear_output(wait=True)
                    
                    # Validate inputs
                    warnings = validate_ui_inputs(ev_slider.value, charging_power_slider.value,
                                                  charging_duration_slider.value,
                                                  charging_window_start.value, charging_window_end.value)
                    if warnings:
                        for w in warnings:
                            print(f"⚠️ {w}")
                        return
                    
                    try:
                        print("🚀 Starting interactive pipeline run...")
                        print("=" * 50)
                        
                        # Display summary of UI parameters
                        summary_panel.value = f"""
                        <b>Input Summary:</b><br>
                        EVs: {ev_slider.value} | Power: {charging_power_slider.value:.1f} kW | Duration: {charging_duration_slider.value}h<br>
                        Off-peak: {charging_window_start.value}:00 → {charging_window_end.value}:00
                        """
                        display(summary_panel)
                        
                        # Set fixed random seeds for reproducibility
                        np.random.seed(42)
                        torch.manual_seed(42)
                        if torch.cuda.is_available():
                            torch.cuda.manual_seed_all(42)
                        
                        # Reduced windows for validation
                        temp_lookback = 48
                        temp_horizon = 12
                        print(f"Using reduced windows: {temp_lookback}h lookback, {temp_horizon}h forecast")
                        
                        global pipeline_state
                        pipeline_state = PipelineState()
                        
                        # Load dataset
                        if dataset_selector.value == 'opsd':
                            raw_df = load_opsd()
                            raw_target_col = 'Consumption'
                        else:
                            raw_df = load_uci_household()
                            raw_target_col = 'Global_active_power'
                        
                        if raw_df.empty or raw_target_col not in raw_df.columns:
                            print(f"❌ Failed to load {dataset_selector.value} dataset")
                            return
                        
                        df = raw_df.rename(columns={raw_target_col: config.TARGET_COLUMN})
                        print(f"✅ Loaded {dataset_selector.value} dataset: {df.shape}")
                        
                        processed_df, scaler = preprocess_for_model(df, config.TARGET_COLUMN)
                        if processed_df.empty:
                            print("❌ Preprocessing failed")
                            return
                        featured_df = build_features(processed_df, f"{config.TARGET_COLUMN}_scaled")
                        if featured_df.empty:
                            print("❌ Feature engineering failed")
                            return
                        print(f"✅ Data preprocessed: {featured_df.shape}")
                        
                        # 70/30 train/test split
                        total_samples = len(featured_df)
                        split_point = int(total_samples * 0.7)
                        train_idx = np.arange(0, split_point)
                        val_idx = np.arange(split_point, total_samples)
                        train_data = featured_df.iloc[train_idx]
                        val_data = featured_df.iloc[val_idx]
                        print(f"Train: {len(train_data)} | Val: {len(val_data)}")
                        
                        min_needed = temp_lookback + temp_horizon
                        if len(val_data) < min_needed:
                            print(f"❌ Insufficient validation data: need {min_needed}, have {len(val_data)}")
                            return
                        
                        # Initialize model
                        target_col_scaled = f"{config.TARGET_COLUMN}_scaled"
                        feature_cols = [c for c in featured_df.columns if c not in [config.TARGET_COLUMN, target_col_scaled]]
                        model = TCNTransformerHybrid(input_size=len(feature_cols), output_size=1,
                                                     horizon=temp_horizon, num_quantiles=len(config.QUANTILES))
                        print(f"✅ Model initialized with {len(feature_cols)} features")
                        
                        import copy
                        temp_config = copy.deepcopy(config)
                        temp_config.LOOKBACK_WINDOW = temp_lookback
                        temp_config.FORECAST_HORIZON = temp_horizon
                        
                        # Train model
                        trained_model, history = train_pytorch_model(model, train_data, val_data, temp_config, feature_cols)
                        if trained_model is None:
                            print("❌ Model training failed")
                            return
                        print("✅ Model training completed")
                        
                        # Generate predictions
                        X_val = history.get('X_val', np.array([]))
                        y_val = history.get('y_val', np.array([]))
                        if X_val.shape[0] == 0:
                            print("⚠️ No validation data available for predictions")
                            return
                        device = torch.device("cuda" if config.env_flags.get('CUDA_AVAILABLE', False) else "cpu")
                        trained_model.to(device)
                        trained_model.eval()
                        with torch.no_grad():
                            predictions = trained_model(torch.from_numpy(X_val).to(device)).cpu().numpy()
                        print(f"✅ Generated predictions: {predictions.shape}")
                        
                        p50_idx = config.QUANTILES.index(0.5) if 0.5 in config.QUANTILES else len(config.QUANTILES)//2
                        baseline_scaled = predictions[0, :, p50_idx]
                        if isinstance(scaler, dict):
                            baseline_forecast = baseline_scaled * scaler['std'] + scaler['mean']
                        else:
                            baseline_forecast = scaler.inverse_transform(baseline_scaled.reshape(-1, 1)).flatten()
                        
                        # Forecast index
                        forecast_start_idx = temp_lookback
                        forecast_end_idx = forecast_start_idx + temp_horizon
                        if len(val_data) >= forecast_end_idx:
                            forecast_index = val_data.index[forecast_start_idx:forecast_end_idx]
                        else:
                            start_time = val_data.index[0] if len(val_data) > 0 else pd.Timestamp.now()
                            forecast_index = pd.date_range(start_time, periods=temp_horizon, freq='H')
                        print(f"✅ Forecast index: {len(forecast_index)} periods ({forecast_index[0]} → {forecast_index[-1]})")
                        
                        # Generate EV load
                        ev_profile = generate_ev_load_profile(
                            num_evs=ev_slider.value,
                            charging_power_kw=charging_power_slider.value,
                            charging_duration_h=charging_duration_slider.value,
                            horizon_h=temp_horizon,
                            baseline_index=forecast_index
                        )
                        scenario_data = create_scenario_dataframe(baseline_forecast, forecast_index, ev_profile)
                        if scenario_data.empty:
                            print("❌ Failed to create scenario")
                            return
                        print("✅ Scenario generated successfully")
                        
                        # Run optimizer
                        optimized_load, shifted_ev = greedy_load_shifter(
                            scenario_data['baseline_load'],
                            scenario_data['ev_load'],
                            charging_window=(charging_window_start.value, charging_window_end.value)
                        )
                        scenario_data['optimized_total_load'] = optimized_load
                        scenario_data['shifted_ev_load'] = shifted_ev
                        print("✅ Optimization completed")
                        
                        # Report
                        report_metrics = generate_advanced_report(scenario_data)
                        if report_metrics:
                            print("\n" + "="*50)
                            print("📊 INTERACTIVE RUN SUMMARY")
                            print("="*50)
                            print(f"Dataset: {dataset_selector.value.upper()}")
                            print(f"Model Config: {temp_lookback}h lookback, {temp_horizon}h forecast")
                            print(f"EVs: {ev_slider.value} | Power: {charging_power_slider.value:.1f} kW | Duration: {charging_duration_slider.value}h")
                            print(f"Off-peak Window: {charging_window_start.value}:00 → {charging_window_end.value}:00")
                            print("="*50)
                        else:
                            print("⚠️ Report generation had issues but optimization completed")
                        
                    except Exception as e:
                        print(f"❌ Error during interactive run: {str(e)}")
                        import traceback
                        print(traceback.format_exc())
            
            run_button.on_click(on_run_button_clicked)
            
            # Layout
            parameter_controls = widgets.VBox([
                widgets.HTML("<h3>Dataset Selection</h3>"),
                dataset_selector,
                widgets.HTML("<h3>EV Scenario Parameters</h3>"),
                widgets.HBox([ev_slider, charging_power_slider, charging_duration_slider]),
                widgets.HTML("<h3>Optimization Settings</h3>"),
                widgets.HBox([charging_window_start, charging_window_end]),
                widgets.HTML("<small>Off-peak window: start hour → end hour (next day if end < start)</small>")
            ])
            
            interface = widgets.VBox([
                widgets.HTML("<h2>🔋 Smart Grid Interactive Dashboard</h2>"),
                widgets.HTML("<p><em>70/30 train/test split with 48h lookback + 12h forecast</em></p>"),
                parameter_controls,
                run_button,
                output_area
            ])
            
            return interface
        
        except Exception as e:
            print(f"Error creating interactive interface: {str(e)}")
            return widgets.HTML("<p>Error: Could not create interactive interface</p>")
    
    print("--- Interactive User Interface ---")
    try:
        ui = create_interactive_interface()
        if ui is not None:
            display(ui)
            print("✅ Interactive interface ready. Adjust parameters and click 'Run Full Pipeline'.")
        else:
            print("❌ Failed to create interactive interface")
    except Exception as e:
        print(f"Error displaying interface: {str(e)}")

else:
    print("⚠️ Interactive UI not available - ipywidgets not found")
    print("💡 Install with: pip install ipywidgets")
    print("📝 Alternative: Use CLI or run cells manually")

--- Interactive User Interface ---


✅ Interactive interface ready. Adjust parameters and click 'Run Full Pipeline'.
